In [7]:
from pycocotools.coco import COCO
import os
import json
import pandas as pd
import numpy as np

In [8]:
# class_full_name_list = ['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
# class_short_name_list = ['General', 'Paper', 'Pack', 'Metal', 'Glass', 'Plastic', 'Foam', 'Bag', 'Battery', 'Cloth']
# class_color_list = [
#     [250,0,50], 
#     [0, 255, 0], 
#     [0, 180, 80], 
#     [185, 185, 185], 
#     [100, 100, 100], 
#     [200, 50, 150], 
#     [50, 150, 200], 
#     [50, 200, 150], 
#     [200, 200, 200], 
#     [255,255,255], 
# ]
# class_color_list = [(np.array(RGB)/255).tolist() for RGB in class_color_list]
# boxform = [{'boxstyle': 'round', 'ec': RGB, 'fc': RGB, 'alpha': 0.4} for RGB in class_color_list] # 'fc': np.clip(np.array(RGB)*10, 0., 1.)

In [48]:
data_dir = '/opt/ml/detection/dataset'
anno_source = '/opt/ml/detection/dataset/train.json'

custom_json_dir = '/opt/ml/detection/teambase/custom_annos'
extension = '.json'
file_name = 'custom'

In [18]:
with open(anno_source) as json_file:
    source_anns = json.load(json_file)

print(source_anns.keys())
source_info = source_anns['info']
source_licenses = source_anns['licenses']
source_images = source_anns['images']
source_categories = source_anns['categories']
source_annotations = source_anns['annotations']
# print(source_info)

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])


In [19]:
source_images[:1]

[{'width': 1024,
  'height': 1024,
  'file_name': 'train/0000.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2020-12-26 14:44:23',
  'id': 0}]

In [20]:
df_images_origin = pd.json_normalize(source_images)
df_anno_origin = pd.json_normalize(source_annotations)
df_anno_origin.head()

,image_id,category_id,area,bbox,iscrowd,id
0,0,0,257301.66,"[197.6, 193.7, 547.8, 469.7]",0,0
1,1,3,10402.56,"[0.0, 407.4, 57.6, 180.6]",0,1
2,1,7,26259.36,"[0.0, 455.6, 144.6, 181.6]",0,2
3,1,4,69096.17,"[722.3, 313.4, 274.3, 251.9]",0,3
4,1,5,24164.58,"[353.2, 671.0, 233.7, 103.4]",0,4


In [21]:
df_anno = df_anno_origin
df_anno[["X","Y","W","H"]] = df_anno['bbox'].tolist()
df_anno.drop(columns='bbox', inplace=True)
df_anno.head()

,image_id,category_id,area,iscrowd,id,X,Y,W,H
0,0,0,257301.66,0,0,197.6,193.7,547.8,469.7
1,1,3,10402.56,0,1,0.0,407.4,57.6,180.6
2,1,7,26259.36,0,2,0.0,455.6,144.6,181.6
3,1,4,69096.17,0,3,722.3,313.4,274.3,251.9
4,1,5,24164.58,0,4,353.2,671.0,233.7,103.4


In [22]:
query = df_anno

In [44]:
select_imgs, select_annos = query['image_id'].unique(), query['id'].unique()
out_images = df_images_origin[df_images_origin['id'] == select_imgs].to_dict('records')
out_annos = df_anno_origin[df_anno_origin['id'] == select_annos].to_dict('records')

In [ ]:
out_info = source_info
out_licenses = source_licenses
out_images = source_images
out_categories = source_categories
out_annotations = source_annotations

out_images = out_images
out_annotations = out_annos

custom_json = {
                'info' : out_info, 
                'licenses' : out_licenses, 
                'images' : out_images, 
                'categories' : out_categories, 
                'annotations' : out_annotations
            }

out_path = os.path.join(custom_json_dir, file_name+extension)   
with open(out_path, 'w') as outfile:
    json.dump(custom_json, outfile)